In [4]:
with open('text.txt','r',encoding='utf-8') as file:
    full_text = file.read()

In [5]:
import yake
kw_extractor = yake.KeywordExtractor(top=10, stopwords=None)
keywords = kw_extractor.extract_keywords(full_text)
for kw, v in keywords:
  print("Keyphrase: ",kw, ": score", v)

Keyphrase:  Metaverse : score 0.004252830557097963
Keyphrase:  world : score 0.04342778010943234
Keyphrase:  building the metaverse : score 0.04839690603725169
Keyphrase:  true metaverse : score 0.04902767005246639
Keyphrase:  building : score 0.05291218186749086
Keyphrase:  virtual world : score 0.07074222920125948
Keyphrase:  metaverses : score 0.07442453474921437
Keyphrase:  building individual metaverses : score 0.08945073236420895
Keyphrase:  Metaverse projects : score 0.09732818797747453
Keyphrase:  connect : score 0.10031024334114362


In [6]:
from multi_rake import Rake
rake = Rake()
keywords = rake.apply(full_text)
print(keywords[:10])

[('draft industry standards', 9.0), ('devoting huge amounts', 9.0), ('inspiring multiple projects', 8.5), ("metaverse's ultimate mission", 8.5), ('big companies mentioning', 8.0), ('fewer physical limitations', 7.5), ('current technology capacity', 7.333333333333334), ('developing metaverse projects', 7.147058823529411), ('greater virtual world', 6.527777777777778), ('building individual metaverses', 6.25)]


In [7]:
from summa import keywords
TR_keywords = keywords.keywords(full_text, scores=True)
print(TR_keywords[0:10])

[('metaverse', 0.40785769087075036), ('metaverses', 0.40785769087075036), ('current technology', 0.15527415967627317), ('world', 0.14825695809210473), ('currently', 0.1383509033245286), ('company', 0.13345806522525172), ('standards', 0.13153277127661986), ('standard', 0.13153277127661986), ('official', 0.12452671168461936), ('officially', 0.12452671168461936)]


In [8]:
from keybert import KeyBERT
kw_model = KeyBERT(model='all-mpnet-base-v2')

In [11]:
keywords = kw_model.extract_keywords(full_text, 

                                     keyphrase_ngram_range=(1, 3), 

                                     stop_words='english',

                                     highlight=False,

                                     top_n=2)

keywords_list= list(dict(keywords).keys())
print(keywords_list)

['metaverse definitely anticipated concepts internet', 'metaverse definitely anticipated concepts internet web3']
